# Downstream Analysis Demo
Examples of how to load the data produced by this pipeline for downstream analysis

## Setup

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from caiman.source_extraction.cnmf.cnmf import load_CNMF
import numpy as np
from pathlib import Path
import pynapple as nap

In [ ]:
session_dir = Path("/media/toor/Elements/aging_mice/sub-Mouse1636/ses-20240530T130237")
dlc_results_hdf5_path = session_dir / "dlc" / "behaviorLinear2024-05-30T13_02_37DLC_resnet50_linearMay29shuffle1_100000_filtered.h5"
caiman_results_path = session_dir / "caiman" / "caiman_results.hdf5"
timestamp_path = session_dir / "timestamps2024-05-30T13_02_37.csv"


## Load Pose Information

First, load the filtered dlc outputs from their .hdf5 file:

In [ ]:


position = pd.read_hdf(dlc_results_hdf5_path, key='df_with_missing') # key='df_with_missing' is the default key
position.head()


Next, load the Caiman outputs using Caiman's `load_CNMF` function:

In [ ]:
cnmf_model = load_CNMF(str(caiman_results_path))
print(f"Total number of identified component traces by timepoints:", cnmf_model.estimates.F_dff.shape)
print("Indices of identified \"good\" components:", cnmf_model.estimates.idx_components.shape)

Finally, load the experiment timestamps and convert them to floats:

In [ ]:
raw_timestamps = pd.read_csv(timestamp_path, header=None, names=["timestamps"])
print(raw_timestamps.head())
timestamps = pd.to_datetime(raw_timestamps.timestamps).map(lambda x: x.timestamp())
print(timestamps)


## Analysis demo

Subset the `F_dff` values using the \"good\" indices. After that, we'll have an array of shape (good components x timeponts).

In [ ]:
F_dff_good = cnmf_model.estimates.F_dff[cnmf_model.estimates.idx_components]
F_dff_good.shape

Specialized datastructures can make working with timeseries data a lot easier. Here's an example of how to work with Pynapple, a neuroscience oriented timeseries analysis package. We'll start by loading our data into Pynapple TsdFrame structures.

In [ ]:
transients = nap.TsdFrame(timestamps.values, F_dff_good.T)
transients

And plot a sample fluorescence trace:

In [ ]:
plt.figure(figsize=(6, 2))
plt.plot(transients[0:2000,100], linewidth=3)
plt.xlabel("Time (s)")
plt.ylabel("Fluorescence")
plt.show()

There are also some utilities for computing tuning curves.To demonstrate that, we'll look at how well a single neuron is tuned to the mouse's position on the linear track. We'll use raw position of the miniscope (x, y) in pixels, but more sophisticated selections of features and timepoints should probably be used when you do this.

In [ ]:
# Multiindexing is funky and worth looking into if this line is weird
miniscope_position = position["DLC_resnet50_linearMay29shuffle1_100000", "objectA"]
miniscope_position = miniscope_position.drop(columns=["likelihood"])
miniscope_position = nap.TsdFrame(timestamps.values, miniscope_position.values, columns=miniscope_position.columns)
miniscope_position

In [ ]:
tuning_curves = nap.compute_2d_tuning_curves_continuous(transients, miniscope_position, 50)


In [ ]:
tuning = tuning_curves[0][100]
x_labels, y_labels = tuning_curves[1]

# Plotting the heatmap
plt.figure(figsize=(8, 6))
plt.imshow(tuning, cmap='viridis', aspect='auto')

# Adding x and y labels
plt.xticks(ticks=np.arange(len(x_labels)), labels=x_labels, rotation=90)
plt.yticks(ticks=np.arange(len(y_labels)), labels=y_labels)

# Adding a color bar
plt.colorbar()

# Adding labels and title
plt.xlabel('X Axis (Pixels)')
plt.ylabel('Y Axis (Pixels)')
plt.title('Tuning of component 100 to the position of the miniscope')

# Displaying the plot
plt.show()